In [1]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers, alpha=0.1):
        self.W = []
        self.layers = layers
        self.alpha = alpha
        
        # start looping from the index of the first layer but stop 
        # before we reach the last 2 layers
        for i in range(0, len(layers) -2):
            
            # randomply init weight matrix connecting the no of 
            # node in each respective layer together adding extra 
            # node for bias
            w = np.random.randn(layers[i] + 1, layers[i+1] + 1)
            self.W.append(w/np.sqrt(layers[i]))
            
            # last 2 layers are special case where input 
            # connections need bias term but the output doesnt
            
            w = np.random.randn(layers[-2]+1, layers[-1])
            self.W.append(w/np.sqrt(layers[-2]))
            
    def __repr__(self):
        # return a string that represents network architecture
        return f"NeuralNetwork {'-'.join(str(l) for l in self.layers)}"
    
    def sigmoid(self, x):
        # return sigmoid activation for given input
        return 1.0/(1+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1-x)
    
    def fit(self, X: np, y: np, epochs=1000, displayUpdate=1000):
        # insert a column of 1 as a last entry in the feature matrix
        X = np.c_[X, np.ones((X.shape[0]))]

        # loop over the epochs
        for epoch in range(epochs+1):
            # loop over the individual data point and train our net
            for x, target in zip(X, y):
                self.fit_partial(x, y)
                
            # check to see if to display update
            if epochs%displayUpdate == 0:
                loss = self.calculate_loss(X, y)
                print(f"[INFO] epoch ={epoch}, loss ={loss}")
            
    def fit_partial(self, x, y):
        
        # construct a list of output activations for each layer as
        # our data point flows thru the network. the first 
        # activation is just the input
        A = np.atleast_2d(x)
        
        #FEEDFORWARD
        for layer, weight_matrix in enumerate(self.W):
            net = A[layer].dot(weight_matrix)
            out = self.sigmoid(net)
            A.append(out)
            
        #BACKPROP
        error = A[-1] - y
    
    def predict(self, X, addBias=True):
        # predict function sends predition for test X
        # AFTER the weights of our neural network have become 
        # optimised from the fit function on the train dataset
        
        p = np.atleast_2d(X)
        
        # check to see if bias col to be added
        if addBias:
            p = np.c_[p, np.ones((p.shape[0]))]
            
        # loop over the layers in the network
        for weight_matrix in self.W:
            p = self.sigmoid(p.dot(weight_matrix))
        
        # convert probabilities to binary labels
        p [p >= 0.5] = 1    
        p [p < 0.5] = 0    
        
        return p   
    
    def calculate_loss(self, X, targets):
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, False)
        loss = 0.5 * np.sum((predictions-targets)**2)
        return loss
     

            
